# Week 04: a data structure as an object

## Reading

- [Objects](https://learning.oreilly.com/library/view/introducing-python-3rd/9781098174392/ch11.html) from Bill Lubanovic's _Introducing Python,_ 3rd edition (free access on the O'Reilly platform using your LUC email).
- [Classes](https://docs.python.org/3/tutorial/classes.html) from the official Python tutorial.
- [Object-oriented design](https://learning.oreilly.com/library/view/python-object-oriented-programming/9781836642596/text/ch01.xhtml#chapter-1-object-oriented-design) from Steven F. Lott and Dusty Phillips's _Python Object-Oriented Programming,_ (free access on the O'Reilly platform using your LUC email).

- [Readability counts](https://peps.python.org/pep-0008/)! This is the _official_ style guide for Python. The document offers tremendous insight to the programming language. This is a **must read** if you plan to write code in Python (or to supervise AI writing code for you) after this course.


In [ ]:
class Character:
    """A class to represent a character in a show."""

    def __init__(self, first_name: str, last_name: str, role: str):
        """Initialize a Character object with the specified first name,
        last name, and role."""
        self.__first_name = first_name
        self.__last_name = last_name
        self.__role = role

    def __str__(self) -> str:
        """Returns a string representation of the Character object. Specifically,
        the string consists of the first letter of the first name, last name,
        and role concatenated together.

        # Single statement with conditional expressions (ternary operators)
        # Elegant but a bit dense.

        return (
            ("" if self.__first_name == "" else self.__first_name[0])
            + ("" if self.__last_name == "" else self.__last_name[0])
            + ("" if self.__role == "" else self.__role[0])
        )
        """
        # Rewrite this method using plain if statements
        string_representation = ""
        if self.__first_name != "":
            string_representation = string_representation + self.__first_name[0]
        if self.__last_name != "":
            string_representation = string_representation + self.__last_name[0]
        if self.__role != "":
            string_representation = string_representation + self.__role[0]
        return string_representation

    def __repr__(self) -> str:
        """Returns a string representation of the Character object for debugging"""
        return self.__str__()

    def get_first_name(self) -> str:
        """Accessor for the first name of the character."""
        return self.__first_name

    @property
    def first_name(self):
        return self.__first_name

    @first_name.setter
    def first_name(self, first_name: str) -> None:
        self.__first_name = first_name

    def set_first_name(self, first_name: str) -> None:
        self.__first_name = first_name

    def get_last_name(self) -> str:
        """Accessor for the last name of the character."""
        return self.__last_name

    def get_role(self) -> str:
        """Accessor for the role of the character."""
        return self.__role

In [ ]:
class Cast:
    """A class to represent the cast of a show, consisting of multiple
    Character objects."""

    def __init__(self, title: str):
        # The title of the show being represented
        self.__title = title
        # A list of Character objects
        self.__underlying = []

    def __len__(self) -> int:
        """Return the number of characters in the show. This allows
        the use of len() on Cast objects."""
        return len(self.__underlying)

    def __bool__(self) -> bool:
        """Return True if there is at least one character in the
        show, False otherwise. This allows the use of bool() on
        Cast objects."""
        return len(self.__underlying) > 0

    def add_character(self, first_name: str, last_name: str, role: str) -> None:
        """Add a new character to the show."""
        # First create a new Character object, then append it to the
        # object's underlying list. The following two steps can be
        # done in one step, but for illustration purposes they are
        # shown separately here.
        new_character = Character(first_name, last_name, role)
        self.__underlying.append(new_character)

    def __contains_by(self, target: str, getter) -> bool:
        """Return True if any character's getter() equals target.
        Here's something wonderful about Python: we can pass functions
        as arguments to other functions. In this case, we can pass in
        any of the getter methods defined in the Character class
        (get_first_name, get_last_name, or get_role) and use that
        to compare against the target string. This allows us to
        avoid duplicating code for each of the three "contains" methods.

        This method is private (indicated by the leading double underscore)
        because it is only intended to be used internally by the other
        "contains" methods."""
        # Prepare to iterate through the underlying list
        i = 0
        # Assume we wont find what we are looking for
        found = False
        # Iterate through the list until we either find a match
        # or run out of items to check
        while i < len(self.__underlying) and not found:
            # Check if the current character's getter() matches target
            # Note that we call the getter function passed in as an argument
            # rather than trying to access it as an attribute of the Character
            # object. If we get a match, variable found becomes True and
            # the loop will exit.
            found = getter(self.__underlying[i]) == target
            # Move to the next character in the list
            i += 1
        # Done searching, return whether we found a match
        return found

    def contains_first_name(self, first_name: str) -> bool:
        """Return True if any character has the specified first name.
        This is a public method that uses the private __contains_by()
        method to perform the actual search."""
        return self.__contains_by(first_name, Character.get_first_name)

    def contains_last_name(self, last_name: str) -> bool:
        """Return True if any character has the specified last name.
        This is a public method that uses the private __contains_by()
        method to perform the actual search."""
        return self.__contains_by(last_name, Character.get_last_name)

    def contains_role(self, role: str) -> bool:
        """Return True if any character has the specified role.
        This is a public method that uses the private __contains_by()
        method to perform the actual search."""
        return self.__contains_by(role, Character.get_role)

    # Constant string template for the report header - used in report()
    # In compliance with the "no magic value" principle
    _REPORT_HEADER = 'There are {} characters in your data about "{}"'

    def report(self) -> str:
        """Generate a nicely formatted report of all characters in the show."""
        output = self._REPORT_HEADER.format(len(self.__underlying), self.__title)
        for character in self.__underlying:
            output += f"\n\t{character.get_first_name()} {character.get_last_name()} - {character.get_role()}"
        return output

---

## Enhancing the `Cast` object

In addition to collecting information about a show's `Character`s we decide to collect information about the actors who play these roles. For example, going back to tracking just strings, we want something like:

```text
[
    [ "David", "Harbour",  "Jim", "Hopper", "Helicopter Parent"], ...
    # ------------------   ------------------------------------
    # Actor name strings   Character and role
]
```

With the `Character` object, the underlying list would look like:

```text
[
    [ "David", "Harbour",  Character("Jim", "Hopper", "Helicopter Parent")], ...
    # ------------------   -----------------------------------------------
    # Actor name strings   Character object
]
```

And if we have an object for the character of a show, why not create an object for the actor as well, so:

```text
[
    [ Actor("David", "Harbour"),  Character("Jim", "Hopper", "Helicopter Parent")], ...
    # -------------------------   -----------------------------------------------
    # Actor object                Character object
]
```

And in UML representation:

![](https://raw.githubusercontent.com/lgreco/images/refs/heads/main/data_structures/super_class_redundant.png)

Noticing the similarities between `Character` and `Actor`, we create a **superclass** to factor out the shared characteristics. `Actor` and `Character` are now **derived classes** _inheriting_ fields and methods from `Person`.

![](https://raw.githubusercontent.com/lgreco/images/refs/heads/main/data_structures/super_class_inherited.png)


---

### Implementation


In [15]:
class Person:
    """A super class that lends characteristics to Actors and Characters"""

    def __init__(self, first_name: str, last_name: str):
        """Instantiate a Person object with the given first and last names."""
        self.__first_name = first_name
        self.__last_name = last_name

    # Plain accessors
    def get_first_name(self) -> str:
        return self.__first_name

    def get_last_name(self) -> str:
        return self.__last_name

    # String representation and its constants
    __FNU = "First Name Unknown"
    __LNU = "Last Name Unknown"

    def __str__(self) -> str:
        """A string representation suitable for user-facing printing"""
        return (
            f"{self.__FNU if self.__first_name=="" else self.__first_name} "
            + f"{self.__LNU if self.__last_name=="" else self.__last_name}"
        )

    def __repr__(self) -> str:
        """A string representation suitable for developer-facing printing."""
        return f"{self.__class__.__name__}: {self.__str__()}"


class Actor(Person):  # <--- Person is a "parameter" for Actor
    """Actor objects are just plain Persons -- the pass statement below
    signals that we don't need anything more than what the superclass Person
    already provides
    """

    pass


class Character(Person):  # <--- Person is a "parameter" for Character
    """Character objects extend the superclass Person by adding one more
    attribute: a description of the character's role.
    """

    def __init__(self, first_name: str, last_name: str, role: str):
        # First instantiate a Person object by invoking the __init__
        # method of the superclass (Person)
        super().__init__(first_name, last_name)
        # Τhen initialize the role field for the δerived object (Character)
        self.__role = role

    # Plain accessor for the additional field
    def get_role(self) -> str:
        return self.__role

In [16]:
hf = Actor("Harrison", "Ford")
ij = Character("Indiana", "Jones", "Cool Prof.")

In [17]:
silly = [hf, ij]
for i in range(len(silly)):
    print(silly[i])

Harrison Ford
Indiana Jones


In [18]:
for item in silly:
    print(repr(item))

Actor: Harrison Ford
Character: Indiana Jones
